In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import re
import streamlit as st

In [17]:
file_path = r"C:\Users\Naomi Natasya\Documents\streamlit-category-product\data\gabungan_data20-24.xlsx"
data = pd.read_excel(file_path)

In [3]:
data

,Nama Produk,Kategori Produk Baru Final
0,Blood Pressure Monitor/ Tensimeter Digital Pow...,Blood Pressure Monitor
1,Non Absorbable Blu Polypropylene Monofilament ...,Suture
2,Nasal Oxygen Cannula Adult,Nasal Cannula
3,"Surgical Mask Earloop, 3 Ply",Face Mask
4,Oxygen Mask Non Rebreathing Anak,Oxygen Mask
...,...,...
245881,ZYBIO Urinalysis Strip (Dry Chemistry Method) ...,Urinalysis Test Strip
245882,ZYBIO Z3 Hematology Analyzer And Accessories +...,Reagent
245883,ZYBIO Z3 Hematology Analyzer And Accessories +...,Reagent
245884,ZYBIO Z3 Hematology Analyzer And Accessories +...,Reagent


In [18]:
print(data.isnull().sum())

Nama Produk                   0
Kategori Produk Baru Final    0
dtype: int64


In [ ]:

import re

def clean_product_name(name):
    if isinstance(name, str):
        # Ganti simbol '/', '-', dan 'x'/'X' dengan spasi
        name = re.sub(r'[-/xX]', ' ', name)
        
        # Hapus angka
        name = re.sub(r'\d+', '', name)
        
        # Hapus karakter selain huruf dan spasi
        name = re.sub(r'[^a-zA-Z\s]', '', name)

        # Ubah ke huruf kecil
        name = name.lower()
        
        # Hapus kata yang hanya terdiri dari 1 huruf
        name = ' '.join([word for word in name.split() if len(word) > 1])

        # Hapus spasi berlebih
        name = re.sub(r'\s+', ' ', name).strip()
        
    return name


# Menerapkan fungsi pembersihan pada kolom 'Nama Produk'
data['Nama Produk'] = data['Nama Produk'].apply(clean_product_name)
# Menghapus baris yang memiliki nilai kosong (NaN) di kolom 'Nama Produk' dan 'Kategori Produk'
data_cleaned = data.dropna(subset=['Nama Produk', 'Kategori Produk Baru Final'])


In [20]:
# Mencetak baris yang memiliki nilai NaN (jika ada)
nan_data = data_cleaned[data_cleaned['Nama Produk'].isna() | data_cleaned['Kategori Produk Baru Final'].isna()]
print("Baris dengan NaN:\n", nan_data)

Baris dengan NaN:
 Empty DataFrame
Columns: [Nama Produk, Kategori Produk Baru Final]
Index: []


In [21]:
data_cleaned

,Nama Produk,Kategori Produk Baru Final
0,blood pressure monitor tensimeter digital powe...,Blood Pressure Monitor
1,non absorbable blu polypropylene monofilament ...,Suture
2,nasal ygen cannula adult,Nasal Cannula
3,surgical mask earloop ply,Face Mask
4,ygen mask non rebreathing anak,Oxygen Mask
...,...,...
245881,zybio urinalysis strip dry chemistry method fu,Urinalysis Test Strip
245882,zybio hematology analyzer and accessories reagent,Reagent
245883,zybio hematology analyzer and accessories reagent,Reagent
245884,zybio hematology analyzer and accessories reag...,Reagent


In [28]:
data_cleaned.to_excel(r"C:\Users\Naomi Natasya\Downloads\datacleaned.xlsx")

In [25]:
# Filter hanya kategori yang punya ≥ 30 data
kategori_counts = data_cleaned['Kategori Produk Baru Final'].value_counts()
kategori_terpilih = kategori_counts[kategori_counts >= 30].index
data_filtered = data_cleaned[data_cleaned['Kategori Produk Baru Final'].isin(kategori_terpilih)].reset_index(drop=True)


In [30]:
# Label Encoding
label_encoder = LabelEncoder()
data_filtered['Kategori Produk Encoded'] = label_encoder.fit_transform(data_filtered['Kategori Produk Baru Final'])


In [31]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data_filtered['Nama Produk'])
y = data_filtered['Kategori Produk Encoded']

In [33]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [34]:
# Menampilkan label encoding kategori produk
print("\nLabel Encoding Kategori Produk Baru Final:")
for label, encoded_label in zip(label_encoder.classes_, range(len(label_encoder.classes_))):
    print(f'{label}: {encoded_label}')


Label Encoding Kategori Produk Baru Final:
Alcohol swab: 0
Ambu Bag: 1
Anesthesia Mask: 2
Annuloplasty Ring: 3
Baby Bed: 4
Bandage: 5
Bed Screen: 6
Bed Sheet: 7
Bed Side Cabinet: 8
Blood Collection Needle: 9
Blood Collection Tube: 10
Blood Glucose/ Chol/ Uric Acid Monitor: 11
Blood Glucose/ Chol/ Uric Acid Test Strip: 12
Blood Pressure Monitor: 13
Blood Transfusion Set: 14
Bone Graft: 15
Bone Plate: 16
Bone Screw: 17
Cannula: 18
Cardiovascular Stent: 19
Cardiovascular stent: 20
Catheter Lubricant: 21
Catheter Mount: 22
Catheters: 23
Chemical indicator strip: 24
Closed Wound Suction Drainage System: 25
Ct Scanner: 26
Defibrillator: 27
Defibrillator Monitor: 28
Dental Strip: 29
Disposal Bag: 30
Drape: 31
Ecg Electrode: 32
Electrocardiograph Machine: 33
Endotracheal Tube: 34
Examination Bed: 35
Examination Lamp: 36
Extension Tube: 37
Face Mask: 38
Fecal Occult Blood Test: 39
Fetal Doppler: 40
Fetal Monitor: 41
Film Viewer: 42
Forceps: 43
Gastrointestinal Stent: 44
Gauze: 45
General Monit

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [36]:
# Model KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

# Melatih model KNN menggunakan data latih
knn.fit(X_train, y_train)

# Menggunakan model untuk memprediksi kategori produk di data uji
y_pred_knn = knn.predict(X_test)
# Evaluasi model KNN menggunakan classification report
print("\nClassification Report - KNN Model:")
print(classification_report(y_test, y_pred_knn))


Classification Report - KNN Model:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       660
           1       0.99      0.99      0.99       278
           2       0.97      0.99      0.98       111
           3       1.00      1.00      1.00        18
           4       0.98      0.98      0.98        55
           5       0.74      0.99      0.85       387
           6       1.00      0.88      0.93        16
           7       1.00      1.00      1.00         6
           8       0.20      1.00      0.33         1
           9       0.87      0.88      0.87        75
          10       0.99      0.99      0.99       641
          11       0.91      0.89      0.90       133
          12       0.99      0.98      0.99      2415
          13       0.99      0.98      0.99       744
          14       0.97      0.99      0.98       255
          15       0.96      1.00      0.98        44
          16       0.97      0.96      0.96  

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def predict_category_with_threshold(product_name, threshold=0.5):
    # Pembersihan nama produk (case folding)
    product_name_cleaned = clean_product_name(product_name)
    
    # Mengonversi produk yang sudah dibersihkan menjadi fitur numerik
    product_name_vectorized = vectorizer.transform([product_name_cleaned])
    
    # Menghitung kesamaan cosine antara produk input dan produk di data pelatihan
    cosine_similarities = cosine_similarity(product_name_vectorized, X_train)
    
    # Mencari nilai maksimum dari cosine similarities (nilai kesamaan tertinggi)
    max_similarity = np.max(cosine_similarities)
    
    # Jika nilai kesamaan lebih rendah dari threshold, anggap tidak ada kecocokan yang valid
    if max_similarity < threshold:
        return ""  # Mengembalikan hasil kosong jika tidak ada kecocokan yang cukup
    
    # Jika kesamaan cukup tinggi, lakukan prediksi kategori menggunakan KNN
    predicted_category_encoded = knn.predict(product_name_vectorized)
    
    # Mengubah hasil prediksi kembali ke kategori asli menggunakan LabelEncoder
    predicted_category = label_encoder.inverse_transform(predicted_category_encoded)
    
    return predicted_category[0]


In [38]:
product_name = "SYRINGE PUMP F04"
predicted_category = predict_category_with_threshold(product_name, threshold=0.5)
print(f"The predicted category for '{product_name}' is: {predicted_category if predicted_category else 'Not Available'}")


The predicted category for 'SYRINGE PUMP F04' is: Syringe Pump


In [37]:
product_name = "NEW KIJANG INNOVA 2.4 G A/T DSL VIN 2021"
predicted_category = predict_category_with_threshold(product_name, threshold=0.5)
print(f"The predicted category for '{product_name}' is: {predicted_category if predicted_category else 'Not Available'}")


The predicted category for 'NEW KIJANG INNOVA 2.4 G A/T DSL VIN 2021' is: Not Available


In [39]:
import joblib

# Setelah Anda melatih model KNN, simpan model dan vectorizer
joblib.dump(knn, 'knn_model.pkl')

['knn_model.pkl']

In [40]:

joblib.dump(vectorizer, 'vectorizer.pkl')



['vectorizer.pkl']

In [41]:

joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [42]:

joblib.dump(X_train, 'X_train.pkl')

['X_train.pkl']

In [43]:

joblib.dump(X_test, 'X_test.pkl')

['X_test.pkl']

In [25]:
pip install pandas joblib scikit-learn openpyxl


In [44]:
import pandas as pd
import joblib
import re
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

# Fungsi pembersihan nama produk
def clean_product_name(name):
    if isinstance(name, str):
        # Menghapus angka dan karakter selain huruf, angka, spasi, dan '/'
        name = re.sub(r'[^a-zA-Z0-9\s/]', '', name)
        # Mengganti '/' dengan spasi
        name = re.sub(r'/', ' ', name)
        # Mengubah menjadi huruf kecil
        name = name.lower()
    return name

# Memuat X_train yang sudah disimpan
X_train = joblib.load('X_train.pkl')

# Memuat model, vectorizer, label encoder yang sudah disimpan
knn = joblib.load('knn_model.pkl')
vectorizer = joblib.load('vectorizer.pkl')
label_encoder = joblib.load('label_encoder.pkl')

# Fungsi prediksi Kategori Produk Baru Final dengan threshold
def predict_category_with_threshold(product_name, knn, vectorizer, label_encoder, X_train, threshold=0.5):
    product_name_cleaned = clean_product_name(product_name)
    
    # Gunakan vectorizer yang sudah dilatih, bukan fit_transform
    product_name_vectorized = vectorizer.transform([product_name_cleaned])  # Hanya transformasi, bukan fit_transform
    
    # Menghitung kesamaan cosine antara produk input dan produk di data pelatihan
    cosine_similarities = cosine_similarity(product_name_vectorized, X_train)
    
    # Mencari nilai maksimum dari cosine similarities (nilai kesamaan tertinggi)
    max_similarity = np.max(cosine_similarities)
    
    # Jika kesamaan lebih rendah dari threshold, anggap tidak ada kecocokan yang valid
    if max_similarity < threshold:
        return "Not Available"  # Tidak ada kecocokan yang cukup
    
    # Prediksi kategori menggunakan model KNN
    predicted_category_encoded = knn.predict(product_name_vectorized)
    predicted_category = label_encoder.inverse_transform(predicted_category_encoded)
    
    return predicted_category[0]

# Fungsi utama untuk memproses file Excel dan melakukan prediksi
def process_and_predict(file_path):
    # Membaca file Excel
    data = pd.read_excel(file_path)

    if 'Nama Produk' not in data.columns:
        print("Data tidak memiliki kolom 'Nama Produk'. Pastikan file Excel memiliki kolom tersebut.")
        return

    # Hapus baris yang kosong
    data = data.dropna(subset=['Nama Produk'])

    # Simpan versi asli sebelum dibersihkan
    data['Nama Produk Asli'] = data['Nama Produk']  # <- Tambahkan ini

    # Buat kolom bersih untuk prediksi saja
    data['Nama Produk Bersih'] = data['Nama Produk'].apply(clean_product_name)

    # Prediksi kategori untuk setiap baris menggunakan kolom bersih
    data['Kategori Produk'] = data['Nama Produk Bersih'].apply(
        lambda name: predict_category_with_threshold(name, knn, vectorizer, label_encoder, X_train, threshold=0.5)
    )

    # Drop kolom bersih, hanya simpan kolom asli dan hasil prediksi
    data = data.drop(columns=['Nama Produk', 'Nama Produk Bersih'])
    data = data.rename(columns={'Nama Produk Asli': 'Nama Produk'})

    # Simpan hasil akhir
    output_file = r"C:\Users\Naomi Natasya\Downloads\predicted_product_categories.xlsx"
    data.to_excel(output_file, index=False)
    print(f"File dengan prediksi kategori telah disimpan sebagai '{output_file}'.")


In [49]:

# Path ke file yang ingin diproses
file_path = r"C:\Users\Naomi Natasya\Downloads\20-22.xlsx"

process_and_predict(file_path)


File dengan prediksi kategori telah disimpan sebagai 'C:\Users\Naomi Natasya\Downloads\predicted_product_categories.xlsx'.
